In [1]:
import cv2
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import os



In [2]:
import cv2
import numpy as np
from sklearn.ensemble import RandomForestClassifier
import os

# 1. Feature extraction function (same as before, but with image check)
def extract_leaf_features(image_path):
    img = cv2.imread(image_path)
    if img is None:
        print(f"[ERROR] Could not read image at path: {image_path}")
        return None
    img = cv2.resize(img, (300, 300))
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    _, thresh = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if not contours:
        return None
    cnt = max(contours, key=cv2.contourArea)
    area = cv2.contourArea(cnt)
    perimeter = cv2.arcLength(cnt, True)
    x, y, w, h = cv2.boundingRect(cnt)
    aspect_ratio = float(w) / h
    extent = float(area) / (w * h)
    hull = cv2.convexHull(cnt)
    hull_area = cv2.contourArea(hull)
    solidity = float(area) / hull_area if hull_area != 0 else 0
    return [area, perimeter, aspect_ratio, extent, solidity]

# 2. Load dataset and train model
def load_dataset(folder_mango, folder_banana, folder_maple):
    X, y = [], []
    
    # Mango leaves
    for f in os.listdir(folder_mango):
        features = extract_leaf_features(os.path.join(folder_mango, f))
        if features:
            X.append(features)
            y.append(0)  # Mango
    
    
    for f in os.listdir(folder_banana):
        features = extract_leaf_features(os.path.join(folder_banana, f))
        if features:
            X.append(features)
            y.append(1)  
    

    for f in os.listdir(folder_maple):
        features = extract_leaf_features(os.path.join(folder_maple, f))
        if features:
            X.append(features)
            y.append(2)  

    return np.array(X), np.array(y)


X, y = load_dataset('dataset/mango', 'dataset/banana', 'dataset/maple')
model = RandomForestClassifier()
model.fit(X, y)


def predict_leaf(image_path):
    features = extract_leaf_features(image_path)
    if features:
        label = model.predict([features])[0]
        if label == 0:
            return "Mango Leaf 🥭"
        elif label == 1:
            return "Banana Leaf 🍌"
        elif label == 2:
            return "Maple Leaf 🍁"
    else:
        return "Could not read or process image."

# Example usage:
test_image = 'test_leaf.jpeg' 
prediction = predict_leaf(test_image)
print(f"Prediction: {prediction}")


Prediction: Banana Leaf 🍌
